<a href="https://colab.research.google.com/github/AnanyaGupta24/PortfolioOptimization/blob/main/Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overall, this code sets up an authenticated connection to Google Drive API that allows the user to access and manage files stored in their Google Drive account programmatically.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


This code is used in Google Colab to link and download datasets from Google Drive into the Colab environment. It downloads CSV files from Google Drive that are stored in different URLs.

Once this code is executed, the files will be downloaded from Google Drive and saved in the Colab environment, allowing the user to access and analyze the data.

In [ ]:
#This part linkes the dataset in the drive to the google colab file
link =  'Link here'
link2 = 'Link here'
link3 = 'Link here'

id = link.split('/')[-2]
id2 = link2.split('/')[-2]
id3 = link3.split('/')[-2]
# id4 = link4.split('/')[-2]

downloaded = drive.CreateFile({'id' : id})
downloaded2 = drive.CreateFile({'id' : id2})
downloaded3 = drive.CreateFile({'id' : id3})

downloaded.GetContentFile('y_dataset.csv')
downloaded2.GetContentFile('LR_Predicted_Prices.csv')
downloaded3.GetContentFile('LR_Actual_prices.csv')


This code reads the S&P 500 index data from a CSV file stored in Google Drive.

In [ ]:
#S&P 500 Data
link = 'link here'
id3 = link.split('/')[-2]
downloaded3 = drive.CreateFile({'id' : id3})
downloaded3.GetContentFile('SP500.csv')
SP500 = pd.read_csv("SP500.csv")


All data sets are converted to pandas data frames, and their indices are set to the date columns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Closing_Prices = pd.read_csv('y_dataset.csv')
Closing_Prices['Date'] =  pd.to_datetime(Closing_Prices['Date'])
Closing_Prices=Closing_Prices.set_index('Date')

#Model: Linear Regression
LR_Predicted_Prices = pd.read_csv('LR_Predicted_Prices.csv')
LR_Predicted_Prices['Date'] =  pd.to_datetime(LR_Predicted_Prices['Date'])
LR_Predicted_Prices=LR_Predicted_Prices.set_index('Date')

LR_Actual_Prices = pd.read_csv('LR_Actual_prices.csv')
LR_Actual_Prices['Date'] =  pd.to_datetime(LR_Actual_Prices['Date'])
LR_Actual_Prices=LR_Actual_Prices.set_index('Date')


This code computes the log returns of the predicted and actual prices for the model as well as the log returns of the actual closing prices.

In [ ]:
#Computing Log Returns
PCA_Predicted_Returns = PCA_Predicted_Prices.apply(lambda x: np.log(x) - np.log(x.shift(1))).iloc[1:]
PCA_Actual_Returns = PCA_Actual_Prices.apply(lambda x: np.log(x) - np.log(x.shift(1))).iloc[1:]

LR_Predicted_Returns = LR_Predicted_Prices.apply(lambda x: np.log(x) - np.log(x.shift(1))).iloc[1:]
LR_Actual_Returns = LR_Actual_Prices.apply(lambda x: np.log(x) - np.log(x.shift(1))).iloc[1:]

Closing_Prices_Returns = Closing_Prices.apply(lambda x: np.log(x) - np.log(x.shift(1))).iloc[1:]


To compute the mean squared error (MSE) between the actual and predicted prices obtained from the PCA model. The MSE is a commonly used metric to evaluate the accuracy of a regression model. It measures the average of the squares of the differences between predicted and actual values. The higher the value of the MSE, the worse the model's performance.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(PCA_Actual_Prices, PCA_Predicted_Prices)

**Creation of Different Lookback and Forward Windows**

In [ ]:
#Mean Returns function
def mean_returns(df, length):
  mu = df.sum(axis = 0)/length
  return mu

This code defines a window generator function that splits a given pandas dataframe into multiple datasets with a lookback period and a forward-looking period. The function takes in the following parameters:

1. dataframe: a pandas dataframe with a DateTimeIndex.
2. lookback: an integer representing the number of months to look back for each window.
3. horizon: an integer representing the number of months to look forward for each window.
4. step: an integer representing the number of months to step forward after each window.
5. cummulative: a boolean value indicating whether the windows should be cumulative or not.

The function first defines a helper function called monthdelta that takes a date and a delta (in months) and returns a new date with the delta months added to the original date. The function handles leap years correctly.

The function then initializes a list to hold the windows and horizons, sets the initial window start to the earliest date in the dataframe, and then loops through the dates in the dataframe. For each date, the function determines the start and end dates for the lookback window and the horizon window using the monthdelta function. It then extracts the corresponding data and appends it to the windows and horizons lists.

The function returns two lists: the windows and the horizons, where each element of the windows list is a dataframe representing a lookback window and each element of the horizons list is a dataframe representing the corresponding horizon window.


In [ ]:
from datetime import timedelta
from dateutil.parser import parse

def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return parse(new_date.strftime('%Y-%m-%d'))


#This part of the code takes in a dataset and splits it into datasets w/ lookback months and forward looking months
def windowGenerator (dataframe, lookback, horizon,
                    step, cummulative = False):

#takes pandas dataframe with DatetimeIndex

    if cummulative:
        c = lookback
        step = horizon

    initial = min(dataframe.index)
    windows = []
    horizons = []

    while initial <= monthdelta(max(dataframe.index), -lookback):
        windowStart = initial
        windowEnd = monthdelta(windowStart, lookback)
        if cummulative:
            windowStart = min(dataframe.index)
            windowEnd = monthdelta(windowStart, c) + timedelta(days=1)
            c += horizon
        horizonStart = windowEnd + timedelta(days=1)
        horizonEnd = monthdelta(horizonStart, horizon)

        lookbackWindow = dataframe[windowStart:windowEnd]
        horizonWindow = dataframe[horizonStart:horizonEnd]

        windows.append(lookbackWindow)
        horizons.append(horizonWindow)

        initial = monthdelta(initial, step)

    return windows, horizons

# **Optimize Function**

The actual_return function takes in actual returns data and a set of portfolio weights, and calculates the portfolio returns and variance.

The scipy_opt function takes in predicted returns data, actual returns data, and two regularization parameters (lam1 and lam2). It performs portfolio optimization using the predicted returns data and the f cost function, which is defined as the negative of the expected portfolio return minus lam1 times the portfolio variance plus lam2 times the L1 norm (the sum of the absolute vector values) of the portfolio weights.
The function uses the SciPy optimization library to find the portfolio weights that maximize this cost function, subject to the constraint that the sum of the weights equals 1.
The resulting portfolio weights are then used to calculate the predicted portfolio returns and variance, as well as the actual portfolio returns, variance, and Sharpe ratio (calculated as the actual portfolio returns divided by the standard deviation of the actual portfolio returns).

In [ ]:
from scipy.optimize import minimize, Bounds, LinearConstraint
from numpy.linalg import norm

def actual_return(actual_returns, w):
  actual_returns = actual_returns
  mean_return = mean_returns(actual_returns, actual_returns.shape[0])
  actual_covariance = actual_returns.cov()

  portfolio_returns = mean_return.T.dot(w)
  portfolio_variance = w.T.dot(actual_covariance).dot(w)
  return portfolio_returns, portfolio_variance


#Input entire predicted returns df, actual returns df, starting date
def scipy_opt(predicted_returns, actual_returns, lam1, lam2):
  mean_return = mean_returns(predicted_returns, predicted_returns.shape[0])
  predicted_covariance = predicted_returns.cov()

  #Cost Function
  def f(w):
    return -(mean_return.T.dot(w) - lam1*(w.T.dot(predicted_covariance).dot(w)) + lam2*norm(w, ord=1))
  #out custom maximises

  #Bounds of Weights
  opt_bounds = Bounds(0, 1)

  #Equality Constraints
  def h(w):
    return sum(w) - 1

  #Constraints Dictionary
  cons = ({
      'type' : 'eq',
      'fun' : lambda w: h(w)
  })

  #Solver
  sol = minimize(f,
                 x0 = np.ones(mean_return.shape[0]),
                 constraints = cons,
                 bounds = opt_bounds,
                 options = {'disp': False},
                 tol=10e-10)


  #Predicted Results
  w = sol.x
  predicted_portfolio_returns = w.dot(mean_return)
  portfolio_STD = w.T.dot(predicted_covariance).dot(w)

  #Actual Results
  portfolio_actual_returns, portfolio_actual_variance = actual_return(actual_returns, w)
  sharpe_ratio = portfolio_actual_returns/np.std(portfolio_actual_variance)

  ret_dict = {'weights' : w,
              'predicted_returns' : predicted_portfolio_returns,
              'predicted_variance' : portfolio_STD,
              'actual_returns' : portfolio_actual_returns,
              'actual_variance' : portfolio_actual_variance,
              'sharpe_ratio': sharpe_ratio}

  return ret_dict

In [ ]:
LR_pred_windows, LR_pred_horizons = windowGenerator(LR_Predicted_Returns, 12, 1, 1)
LR_act_windows, LR_act_horizons = windowGenerator(LR_Actual_Returns, 12, 1, 1)

In [ ]:
LR_scipy_returns= []
LR_scipy_variance = []
LR_scipy_SR = []

#Testing on 5 years of data
for i in range(len(LR_act_horizons)-72,len(LR_act_horizons)-12):
  #Scipy optimize results
  scipy = scipy_opt(LR_pred_horizons[i], LR_act_horizons[i], .5, 2)
  LR_scipy_returns.append(scipy['actual_returns'])
  LR_scipy_variance.append(scipy['actual_variance'])
  LR_scipy_SR.append(scipy['sharpe_ratio'])
  # print("Month " + str(i) + " complete")

In [ ]:
timestamps = []
for i in range(len(LR_act_horizons)-72,len(LR_act_horizons)-12):
  time = LR_act_horizons[i].index[-1]
  timestamps.append(time)

LR_Portfolio_Returns = pd.DataFrame(data = np.array([LR_scipy_returns,LR_scipy_variance,LR_scipy_SR]).T, columns = ['Returns', 'Variance', 'Sharpe Ratio'], index=timestamps)


In [ ]:
## Exporting the Dataset
from google.colab import drive
drive.mount('drive')

LR_Portfolio_Returns.to_csv('LR_Portfolio_Returns.csv')
!cp LR_Portfolio_Returns.csv "drive/My Drive/Machine Learning Project/ML Section Expo

# **Diagnostics**
The calculation for seeing how much the portfolio grows in dollar terms:

Given $100, our equity graph shows how much our portfolio value increase (or decreases)

In [ ]:
import math
LR_equity = [100]

#This is the calculation for the for seeing how much the portfolio grows
for i in range(1,60):
  LR_equity.append(LR_equity[i-1]* math.exp(LR_scipy_returns[i]))


In [ ]:
# Plots line graph b/w years and equity
plt.plot(timestamps, LR_equity, label = "Linear Regression")
plt.title("Equity Graph")
plt.legend()
plt.show();

In [ ]:
print("Linear Regression Ending Equity: " , LR_equity[-1])\

In [ ]:
import math
def metrics(returns):
  sharpe = returns.mean() / returns.std()
  annualized_sharpe = sharpe.item() / math.sqrt(252)

  stdev = returns.std()
  annualized_vol = stdev.item() / math.sqrt(252)


  return {"Annualized Sharpe Ratio": annualized_sharpe,
          "Annualized Volatility": annualized_vol}

In [ ]:
#Annualized info for Linear Regression
metrics(np.array(LR_scipy_returns))